In [6]:
import pandas as pd
import cufflinks as cf
import plotly.offline
import plotly.graph_objs as go  
import mytools.regression as reg
import mytools.date as dt

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Italian intensive care cases 

In [7]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d %b") for v in df['day'])
df.index = a
# display the table
pd.DataFrame(df['intensive care'])



,intensive care
25 Feb,27
26 Feb,35
27 Feb,36
28 Feb,56
29 Feb,64
01 Mar,105
02 Mar,140
03 Mar,166
04 Mar,229
05 Mar,295


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [8]:
x_orig = df['day']
y_orig = df['intensive care']

sigm_model, xp, pxp = reg.fit_sigmoid(x_orig, y_orig, verbose=True)
flex = reg.sigmoid_get_flex(sigm_model)

exp_model, exp_xp, exp_pxp = reg.fit_exponential(x_orig, y_orig, upper=1.25, verbose=True)


`ftol` termination condition is satisfied.
Function evaluations 21, initial cost 1.3451e+00, final cost 5.4330e-04, first-order optimality 4.20e-08.
        Sigmoid model
        x0 = 67.61293339713713
        y0 = 6.308972991552549
        c = 1221.1436534635416
        k = 0.32984684066051917
        asymptot = 1227.452626455094
        flex = 67.61293339713713, 616.8807997233233
        
[0.65       0.44157303 1.        ]
`gtol` termination condition is satisfied.
Function evaluations 14, initial cost 5.2239e+00, final cost 1.3999e-03, first-order optimality 3.99e-09.
            Exponential model
            x0 = 33.51153139953671
            y0 = -48.25364922719594
            k = 0.1907646327943292
            


In [9]:
# preparing table with predictions
d_max = df['day'].max()
d_min = df['day'].min()
f_days = range(d_min - 4, d_max + 4, 1)

f_exp = reg.exponential(exp_model, f_days)
f_sigm = reg.sigmoid(sigm_model, f_days)
f_df = pd.DataFrame({'day' : f_days, 'exp' : f_exp, 'sigm' : f_sigm})
f_df.index = dt.day_of_year_to_string(f_days)

df_final = pd.merge(df, f_df, on='day', how='outer', sort=True)
df_final.index = dt.day_of_year_to_string(f_days)
df_final

,day,intensive care,exp,sigm
21 Feb,51,NaN,-20.142078,11.380580
22 Feb,52,NaN,-14.233740,13.350908
23 Feb,53,NaN,-7.083620,16.080561
24 Feb,54,NaN,1.569274,19.856499
25 Feb,55,27.0,12.040786,25.068937
26 Feb,56,35.0,24.713144,32.243802
27 Feb,57,36.0,40.048910,42.081099
28 Feb,58,56.0,58.607863,55.496205
29 Feb,59,64.0,81.067437,73.656354
01 Mar,60,105.0,108.247443,97.996764


In [10]:
fig = df_final[['exp', 'sigm', 'intensive care']].iplot(theme="white", title="Intensive care cases", size=4, yTitle='case', mode='lines+markers', asFigure=True)
fig.data[2].mode='markers'
fig.data[2].marker.size = 8
fig.show()